## Neural Network Multiple Multiclass

In [ ]:
%run DataSplitting.ipynb
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras.callbacks import EarlyStopping
# %matplotlib widget
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)
# from public_tests import *
# from autils import *
# from lab_utils_softmax import plt_softmax
np.set_printoptions(precision=2)



## Softmax Definition

In [ ]:
def my_softmax(z):
    ### START CODE HERE ###
    ez = np.exp(z)
    a = ez/np.sum(ez)
    ### END CODE HERE ###
    return a

## Model Declaration

In [ ]:
tf.random.set_random_seed(1234) # for consistent results
model = Sequential(
    [
        ### START CODE HERE ###
        tf.keras.layers.InputLayer((X_train.shape[1],)),
        tf.keras.layers.Dense(20, activation="relu", name="L1"),
        tf.keras.layers.Dense(3, activation="softmax", name="L2")
        ### END CODE HERE ###
    ], name = "my_model"
)
# model.compile(
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
# )


In [ ]:
model.summary()

# Examine Layer Weights

In [ ]:
[layer1, layer2] = model.layers
#### Examine Weights shapes
W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")

## Define Loss function

In [ ]:

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001),
)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
#y_train = [Age<8, Age>=8 && Age<15, Age>15]
#y_train_classification=['young','middle-age','old']
#y_train_encoded = [2,0,1]
history = model.fit(
    X_train_scaled, y_train_encoded,
    validation_data=(X_val_scaled, y_val_encoded),
    epochs=30,
    batch_size=32, 
    callbacks=[early_stopping]
)

# Loss Plot

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss with SGD')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
print(history.params)

## Tuning and Evaluation

In [ ]:
# def build_model(num_neurons):
#     model_2 = Sequential([
#         tf.keras.layers.InputLayer((X_train.shape[1],)),
#         tf.keras.layers.Dense(num_neurons, activation="relu", name="L1"),
#         tf.keras.layers.Dense(3, activation="softmax", name="L2")
#     ], name="my_model_2")

#     model_2.compile(
#          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#     )

#     return model_2
# neuron_configs = [5, 10, 20, 30, 50]
# validation_losses = []


# for num_neurons in neuron_configs:
#     model_2 = build_model(num_neurons)
#     #using early stopping to automatically stop training when the validation loss stops improving. This can prevent overfitting. 
#     early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

#     history = model_2.fit(X_train_scaled, y_train_encoded, validation_data=(X_val_scaled, y_val_encoded), epochs=30, batch_size=32, callbacks=[early_stopping])
#     # Evaluate and print performance metrics
#     eval_metrics = model_2.evaluate(X_val_scaled, y_val_encoded)
#     validation_loss = eval_metrics
#     validation_losses.append((num_neurons, validation_loss))

# # Sort and print validation losses
# sorted_losses = sorted(validation_losses, key=lambda x: x[1])
# for neurons, loss in sorted_losses:
#     print(f"Neurons: {neurons}, Validation Loss: {loss}")

## Prediction

In [ ]:

#y_train = [Age<8, Age>=8 && Age<15, Age>15]
#y_train_classification=['young','middle-age','old']
#y_train_encoded = [2,0,1]

age_of_index = X_test_scaled[300]
prediction = model.predict(age_of_index.reshape(1,11))  # prediction
print(f" predicting: \n{prediction}")
print(f" Largest Prediction index: {np.argmax(prediction)}")

## Probability of output with SoftMax

In [ ]:
# prediction_p = tf.nn.softmax(prediction)

# with tf.compat.v1.Session() as sess:
#     prediction_p_np = prediction_p.eval()

# print(f"Probability vector:\n{prediction_p_np}")
# print(f"Total of predictions: {np.sum(prediction_p_np):0.3f}")


# predicted_class = np.argmax(prediction_p_np)
# print(f"Predicted Class Index: {predicted_class}")


## Unmatched Prediction and Real Value

In [ ]:
y_pred=model.predict(X_test_scaled)  
# Get the predicted class labels
predicted_classes = np.argmax(y_pred, axis=1)
# Find indices where predictions don't match true labels
incorrect_indices = np.where(predicted_classes != y_test_encoded)[0]
# Store incorrect predictions for later printing
incorrect_predictions = []
# Print out the ages and corresponding predictions for the incorrect instances
for index in incorrect_indices:
    age = y_test_encoded[index]
    prediction = predicted_classes[index]
    incorrect_predictions.append((index, age, prediction))
    print(f"Index: {index}, True Age: {age}, Predicted Age: {prediction}")
# Calculate accuracy
accuracy = np.sum(predicted_classes == y_test_encoded) / len(y_test_encoded)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print detailed information about incorrect predictions
# print("\nIncorrect Predictions:")
# for index, true_age, predicted_age in incorrect_predictions:
#     print(f"Index: {index}, True Age: {true_age}, Predicted Age: {predicted_age}")

## Confusion Matrix

In [ ]:
conf_matrix = confusion_matrix(y_test_encoded, predicted_classes)
#y_train = [Age<8, Age>=8 && Age<15, Age>15]
#y_train_classification=['young','middle-age','old']
#y_train_encoded = [2,0,1]
# Plot confusion matrix using seaborn
plt.figure(figsize=(4,3))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Reds', cbar=False,
            xticklabels=['middle-age', 'old', 'young'],
            yticklabels=['middle-age', 'old', 'young'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix with SGD')
plt.show()